In [1]:
import pandas as pd

In [34]:
origin_classes = pd.read_csv("../benchmark_classes.csv")

In [10]:
hadoop = origin_classes[origin_classes['project']=='o4_hadoop']

In [12]:
others = origin_classes[origin_classes['project'] !='o4_hadoop']

In [20]:
hadoop_class_list = list(hadoop['class'])

In [163]:
def get_package_list(hadoop_class_list, level=3):
    r = []
    for v in hadoop_class_list:
        v = v.strip()
        tv = v.split(".")
        tl = -level
        if len(tv) <= level:
            tl = len(tv)-1
        tv = tv[0:tl]
        nv = '.'.join(tv)
        r.append(nv)
    return r

def get_groups(origin_classes, hadoop_level=3):
    hadoop = origin_classes[origin_classes['project']=='o4_hadoop']
    others = origin_classes[origin_classes['project'] !='o4_hadoop']
    hadoop_class_list = list(hadoop['class'])
    hadoop_package_list = get_package_list(hadoop_class_list, hadoop_level)
    others_project_list = list(others['project'])
    ret = []
    hadoop_package_map = {}
    for v in set(others_project_list):
        ret.append([v])
    ps = sorted(list(set(hadoop_package_list)), reverse=True)
    for i in range(0, len(hadoop_class_list)):
        if hadoop_package_list[i] not in hadoop_package_map:
            hadoop_package_map[hadoop_package_list[i]] = []
        hadoop_package_map[hadoop_package_list[i]].append(hadoop_class_list[i])
    for b in  ps:
        ret.append(['o4_hadoop', b])
    return ret, hadoop_package_map

def split_data_by_groups(origin_classes, groups,hadoop_package_map):
    ret = []
    for g in groups:
        sd = None
        if len(g) == 1:
            sd = origin_classes[origin_classes['project']==g[0]]
        else:
            sd = origin_classes[origin_classes['class'].isin(hadoop_package_map[g[1]])]
        ret.append([g, len(sd), sd])
    return ret

def genereate_final_data(sdg, origin_classes):
    d = {
        'group': [],
        'classes': [],
        'branch_25':[],
        'branch_mean':[],
        'branch_75':[],
        'statement_25':[],
        'statement_mean':[],
        'statement_75':[],
    }
    for g in sdg:
        if len(g[0]) == 1:
            d['group'].append(g[0][0])
        else:
             d['group'].append(g[0][0] + '-' + g[0][1])
        
        d['classes'].append(g[1])
        describe_data = g[2].describe()
        d['branch_25'].append(int(describe_data['n.branch']['25%']))
        d['statement_25'].append(int(describe_data['n.statement']['25%']))
        d['branch_mean'].append(int(describe_data['n.branch']['mean']))
        d['statement_mean'].append(int(describe_data['n.statement']['mean']))
        d['branch_75'].append(int(describe_data['n.branch']['75%']))
        d['statement_75'].append(int(describe_data['n.statement']['75%']))
        
    
    d['group'].append('all')
    d['classes'].append(len(origin_classes))
    describe_data = origin_classes.describe()
    d['branch_25'].append(int(describe_data['n.branch']['25%']))
    d['statement_25'].append(int(describe_data['n.statement']['25%']))
    d['branch_mean'].append(int(describe_data['n.branch']['mean']))
    d['statement_mean'].append(int(describe_data['n.statement']['mean']))
    d['branch_75'].append(int(describe_data['n.branch']['75%']))
    d['statement_75'].append(int(describe_data['n.statement']['75%']))
        
    return pd.DataFrame(data=d)
        

In [107]:
origin_classes = pd.read_csv("../benchmark_classes.csv")
groups,hadoop_package_map = get_groups(origin_classes,4)
sdg = split_data_by_groups(origin_classes, groups, hadoop_package_map)

In [164]:
fd = genereate_final_data(sdg, origin_classes)

In [165]:
fd

,group,classes,branch_25,branch_mean,branch_75,statement_25,statement_mean,statement_75
0,74_fixsuite,1,74,74,74,631,631,631
1,107_weka,3,257,441,538,1505,3606,4777
2,35_corina,1,55,55,55,282,282,282
3,40_glengineer,1,115,115,115,658,658,658
4,97_feudalismgame,1,788,788,788,4873,4873,4873
...,...,...,...,...,...,...,...,...
82,o4_hadoop-org.apache.hadoop.crypto,1,82,82,82,337,337,337
83,o4_hadoop-org.apache.hadoop,40,62,220,194,133,403,331
84,o4_hadoop-org.apache,65,70,343,158,105,565,295
85,o4_hadoop-org,45,70,151,156,121,271,313


In [166]:
fd.to_csv('class_stat.csv', index=False)